In [1]:
import kagglehub
import pandas as pd
import json

### Import Data


In [2]:
path = kagglehub.dataset_download("mathurinache/math-dataset")

#### Prealgebra

In [11]:
import os

problems = {"problem": [], "level": [], "type": [], "solution": []}

with os.scandir(path + "/MATH/train/") as types:
    for type in types:
        if type.is_dir():
            with os.scandir(type) as entries:
                for entry in entries:
                    if entry.is_file():
                        # print(entry.path)
                        with open(entry.path) as f:
                            data = json.load(f)
                            for key in data:
                                problems[key].append(data[key])

all_levels = pd.DataFrame.from_dict(problems, orient='columns')

train = all_levels[all_levels['level'] == "Level 1"]

,problem,level,type,solution
4,"Find the value of $\frac{5x+9y}{45xy}$, given ...",Level 1,Algebra,We substitute the values for $x$ and $y$ into ...
11,What is the value of $\displaystyle\frac{235^2...,Level 1,Algebra,The key to this problem is noticing that $235^...
13,Compute: $55\times1212-15\times1212$ .,Level 1,Algebra,We have $55 \times 1212 - 15 \times 1212 = 121...
34,"If $f(x) = 3x + 1$, what is $f(3)$?",Level 1,Algebra,We have $f(3) = 3(3) + 1 = \boxed{10}$.
45,The first three terms of an arithmetic sequenc...,Level 1,Algebra,The common difference for this arithmetic sequ...
...,...,...,...,...
7420,What is the remainder when 2007 is divided by 25?,Level 1,Number Theory,Since 25 divides evenly into 2000 and 2007 is ...
7421,"If $n$ is an integer, what is the remainder wh...",Level 1,Number Theory,We see that $(7 - n) + (n + 3) = 10 \equiv 3 \...
7453,For what value of $n$ is the four-digit number...,Level 1,Number Theory,"We know that $18=9\cdot2$, so in order for the..."
7472,What two-digit positive integer is one more th...,Level 1,Number Theory,The least common multiple of these integers is...
